In [4]:
import importlib
import gui
import people
import astar
importlib.reload(gui)
importlib.reload(people)
importlib.reload(astar)
from gui import GUI
from people import PeopleList
import matplotlib.pyplot as plt

In [19]:
import time as real_time  # import real time

# [(start_boundary), (end_boundary), (door), (start_obstacle1), (end_obstacle1), (start_obstacle2), (end_obstacle2)...]
barrier_set = [(0, 0), (27, 17), (26, 8), (24, 4), (25, 7), (24, 10), (25, 13)]
# = [(0, 0), (27, 17), (26, 8), (22, 4), (23, 7)]
# barrier_set = [(0, 0), (27, 17), (26, 8)]  # -> No obstacles
delta_time = 0.005
percent_threshold = 50  # the percentage of people being moved out
initial_people_count = None
recorded_time = False

# Build GUI
gui = GUI()
gui.add_barrier(barrier_set)
gui.update_gui()

# Build PeopleList saving each people object
people_list = PeopleList()
# Build direction matrix
people_list.direction_matrix(barrier_set)
# Assign groups
group_split = {4: 1, 3: 2, 2: 5}
people_list.assign_groups(group_split)
time = 0

# Initial each people
for people in people_list.list:
    gui.add_line(people, people_list.list, people.group_id)
    gui.add_oval(people.loc[0] - people.r, people.loc[1] - people.r,
                 people.loc[0] + people.r, people.loc[1] + people.r, people.id)
gui.update_gui()

initial_people_count = len(people_list.list)

# Move
start_time = real_time.time() 

while people_list.list:
    i = 0
    while i < len(people_list.list):
        gui.del_line(people_list.list[i].group_id)
        gui.del_oval(people_list.list[i].id)

        if people_list.list[i].loc[0] > 1040:  # delete people if it goes out of room
            people_list.list.pop(i)
            continue
        i += 1

    # Check the percentage of people who have moved out
    remaining_people_count = len(people_list.list)
    moved_out_percentage = ((initial_people_count - remaining_people_count) / initial_people_count) * 100

    if not recorded_time and moved_out_percentage >= percent_threshold:
        print(f"Time when {percent_threshold}% of people moved out: {round(time, 3)} seconds") 
        recorded_time = True

    people_list.move(barrier_set, delta_time=delta_time, A=2000, B=-0.08, threshold_ped=1.4, threshold_obs=120,
                     threshold_group=2, q_A=1, q_R=1, beta1=4, beta2=2, beta3=2)  # ped movement

    for people in people_list.list:  # update position
        gui.add_oval(int(people.loc[0]) - people.r,
                     int(people.loc[1]) - people.r, int(people.loc[0]) + people.r,
                     int(people.loc[1]) + people.r, people.id)
        gui.add_line(people, people_list.list, people.group_id)

    time = real_time.time() - start_time  # update time
    gui.update_time(str(round(time, 3)))
    gui.update_gui()

gui.start()


TclError: can't invoke "update" command: application has been destroyed